# Attachment Protocol Example - Alice

This notebook tests the capablity to extend the basic controller to control a custom protocol developed in an aries agent. This protocol allows you to attach a file to a message.
Run this along side [Attachment Protocol Bob](http://localhost:8889/notebooks/attachment.ipynb)

In [19]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


### Intialise a controller 

In [29]:
from om_aries_controller.om_aries_controller import OMAriesController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8028
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
om_controller = OMAriesController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)




### Confirm that an active connection exists

In [30]:
response = await om_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")
    

Results :  [{'created_at': '2020-09-20 05:54:36.122873Z', 'routing_state': 'none', 'their_did': 'WKzwXjVv6258emQfccX3Yr', 'request_id': '70934b96-6be4-43f5-888b-bdf94389f4de', 'invitation_key': '3tp8tJX375c8HmQgEw3uexnjYa366cPgmcNBxHn1BP9E', 'accept': 'manual', 'connection_id': 'e2b975e7-d8d3-464c-98d4-a9a0d49cb202', 'invitation_mode': 'once', 'state': 'active', 'my_did': 'B42T5iUeJ1NZxqecf713eD', 'initiator': 'external', 'updated_at': '2020-09-20 05:54:58.679292Z', 'their_label': 'Bob'}]
Connection : {'created_at': '2020-09-20 05:54:36.122873Z', 'routing_state': 'none', 'their_did': 'WKzwXjVv6258emQfccX3Yr', 'request_id': '70934b96-6be4-43f5-888b-bdf94389f4de', 'invitation_key': '3tp8tJX375c8HmQgEw3uexnjYa366cPgmcNBxHn1BP9E', 'accept': 'manual', 'connection_id': 'e2b975e7-d8d3-464c-98d4-a9a0d49cb202', 'invitation_mode': 'once', 'state': 'active', 'my_did': 'B42T5iUeJ1NZxqecf713eD', 'initiator': 'external', 'updated_at': '2020-09-20 05:54:58.679292Z', 'their_label': 'Bob'}
Active Conne

### Instantiate Listener for the Attachment topic

This is emitted using PyPubSub when the controller receives a atatchment webhook from the agent. This happens everytime the agent receives an attachment.

In [31]:

def attach_handler(payload):
    connection_id = payload['connection_id']
    print("Handle Attachment", payload, connection_id)

attach_listener = {
    "handler": attach_handler,
    "topic": "attachment"
}
loop = asyncio.get_event_loop()
loop.create_task(om_controller.listen_webhooks())
om_controller.register_listeners([attach_listener], defaults=True)

### Send an attachment over DIDcomm to Bob

In this instance a text file is sent, this will be received and saved by the handler in the [Bob notebook](http://localhost:8889/notebooks/attachment.ipynb).

In [32]:
from aiohttp import FormData
data = FormData()
data.add_field('file',
               open('test_file.txt', 'rb'),
               filename='test_file.txt',
               content_type='text/plain')
response = await om_controller.protocol.send_attachment(connection_id, data)
print('Attachment- Alice-> Bob')
print(response)


Attachment- Alice-> Bob
{'thread_id': 'b56bc2fb-b17a-48ca-a06b-93e3327ea4e1'}


Here an image file is sent and is received by the handler in [Bob notebook](http://localhost:8889/notebooks/attachment.ipynb).

In [33]:
import base64
from aiohttp import FormData

data = FormData()
data.add_field('file',
               open('openmined.jpg', 'rb'),
               filename='openmined.jpg',
               content_type='image/jpeg')
response = await om_controller.protocol.send_attachment(connection_id, data)
print('Attachment- Alice-> Bob')
print(response)

Attachment- Alice-> Bob
{'thread_id': 'a5706820-8851-4f92-a6d6-377ecaa28501'}



# End of Tutorial

Be sure to terminate the controller so you can run another tutorial

In [8]:
response = await om_controller.terminate()
print(response)


None
